In [1]:
import json
import os
import time
import pickle

from instagramy import InstagramUser
from instagramy import InstagramHashTag
from instagramy import InstagramPost

from nedima.utils import adls_utils
from nedima.utils import groundswell as gs

In [2]:
secrets = adls_utils.load_secrets()
adls_client = adls_utils.initialize_storage_account_ad()
fs_client = adls_client.get_file_system_client(file_system=secrets['azure']['adls_fs_landing'])

In [3]:
#secrets

In [4]:
while(True):
    try:
        tag_dated, sleep_time = adls_utils.load_inspection_snapshot()
    except:
        tag_dated = InstagramHashTag('surf', sessionid=secrets['instagram']['session_id'])
        sleep_time = 30 + gs.calculate_waiting_time(tag_dated, tag_dated)
        print("[SNAPSHOT] The snapshot couldn't be loaded. A new snapshot will be generated. Some posts may have gotten lost.")
        print("[SNAPSHOT] {} seconds remaining until next inspection.".format(sleep_time))
    
    time.sleep(sleep_time)
    tag_latest = InstagramHashTag('surf', sessionid=secrets['instagram']['session_id'])

    dump_dict = {
        "posts_short" : gs.get_diff_top_posts(tag_latest, tag_dated),
        "posts_full" : gs.get_diff_json(tag_latest, tag_dated, False)
    }

    adls_utils.dump_adls_json(dump_dict, tag_latest, fs_client)

    adls_utils.dump_inspection_snapshot(tag_latest, tag_dated) 

[SNAPSHOT] Next inspection scheduled for 2021-02-14 15:49:21. 63 seconds remaining.
